In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from joblib import dump,load

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
df

,Formula,K_frac,Rb_frac,Cs_frac,MA_frac,FA_frac,Li_frac,Ca_frac,Sr_frac,Ba_frac,...,X_Heat_of_fusion(kJ/mol),A_Heat_of_vap(kJ/mol),B_Heat_of_vap(kJ/mol),X_Heat_of_vap(kJ/mol),A_Electronegativity,B_Electronegativity,X_Electronegativity,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,MAGeCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,334.30,10.20,2.55,2.01,3.16,-0.220,3.022,0.025
1,MAGeBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,334.30,14.73,2.55,2.01,2.96,-0.327,3.323,0.012
2,MAGeI_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7.76,23.85,334.30,20.90,2.55,2.01,2.66,0.330,2.557,0.062
3,MASnCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,290.37,10.20,2.55,1.96,3.16,-0.228,3.182,0.012
4,MASnBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,290.37,14.73,2.55,1.96,2.96,-0.178,3.532,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,FASnBr_0.75Cl_2.25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.73,29.00,290.37,11.33,2.57,1.96,3.11,-1.161,3.882,0.003
295,FAPbBr_0.375Cl_2.625,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.47,29.00,177.90,10.77,2.57,2.33,3.14,-1.039,4.129,0.002
296,FABaI_2.625Br_0.375,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,7.45,29.00,140.20,20.13,2.57,1.00,2.70,-0.549,4.759,0.001
297,MAPbI_1.5Br_1.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,6.53,23.85,177.90,17.82,2.55,2.33,2.81,0.219,3.269,0.013


In [3]:
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
Y

,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,-0.220,3.022,0.025
1,-0.327,3.323,0.012
2,0.330,2.557,0.062
3,-0.228,3.182,0.012
4,-0.178,3.532,0.004
...,...,...,...
294,-1.161,3.882,0.003
295,-1.039,4.129,0.002
296,-0.549,4.759,0.001
297,0.219,3.269,0.013


In [4]:
Y=Y['Band Gap (eV)']
Y

0      3.022
1      3.323
2      2.557
3      3.182
4      3.532
       ...  
294    3.882
295    4.129
296    4.759
297    3.269
298    3.641
Name: Band Gap (eV), Length: 299, dtype: float64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (224, 57) ; y_train's shape is (224,)
X_test's shape is (75, 57) ; y_test's shape is (75,)


In [6]:
##gridsearch and cv select model##
param_grid = {'weights': ['uniform','distance'],
                          'n_neighbors': range(1,10),
                          'algorithm': ['ball_tree','kd_tree','brute'],
                          
                        }

In [7]:
knn = KNeighborsRegressor()
model = GridSearchCV(estimator =knn,scoring='neg_root_mean_squared_error', param_grid = param_grid,cv=5, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train_stand, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('Bg_KNN13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 5 folds for each of 54 candidates, totalling 270 fits
{'mean_fit_time': array([0.00100079, 0.00080009, 0.0007998 , 0.00100179, 0.00090132,
       0.00059929, 0.00099959, 0.00079966, 0.00100026, 0.0010601 ,
       0.00080142, 0.00119939, 0.00120115, 0.00099974, 0.00099978,
       0.00099959, 0.00110087, 0.00100017, 0.00159922, 0.00159936,
       0.00099931, 0.0010005 , 0.00119953, 0.00140038, 0.00140061,
       0.00120029, 0.00100322, 0.00140057, 0.00140257, 0.00120192,
       0.00140057, 0.00140009, 0.00180006, 0.00100174, 0.00100017,
       0.00140004, 0.00099721, 0.00099978, 0.00019965, 0.00070138,
       0.00060387, 0.00040007, 0.00079932, 0.00020156, 0.00100036,
       0.00030107, 0.00070176, 0.00100465, 0.00060091, 0.0008009 ,
       0.00060024, 0.0006    , 0.00080037, 0.0008007 ]), 'std_fit_time': array([1.14440918e-06, 7.48315494e-04, 3.99899863e-04, 2.80646114e-06,
       8.01250217e-04, 4.89318308e-04, 2.23961231e-06, 3.99830644e-04,
       1.02934995e-06, 1.48936298e-